<a href="https://colab.research.google.com/github/m-mehdi/Python101/blob/master/Apache_Spark_06_Clustering_PNB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="images/cads-logo.png" style="height: 100px;" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%" align=right>

# Clustering
In clustering, we are going to see if there are natural grouping among the data. So, for example, let's take a look at the utilization data, and see if we can divide this data set into three groups that logically come together. So to do that, we need the Apache Spark Machine Learning package. 

In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

     |████████████████████████████████| 204.2MB 62kB/s 
     |████████████████████████████████| 204kB 34.5MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=06b44c487f9aafb715ff9c45eaca6184952e8692145ada6774fb1a1ff4264f43
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
import os
MAIN_DIRECTORY = os.getcwd()
file_path =MAIN_DIRECTORY+"/Data/utilization.json"
df_util = spark.read.format("json").load(file_path)

In [4]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

Now, we would like to group data based on the CPU utilization, free memory, and session count. Spark MLLib works with something called a vector. A vector is basically like an array or single data structure that holds all the values from a particular row that the ML algorithm will be looking at. So in our case, we are going to look at only three columns, `cpu_utilization`, `free_memory`, and `session_count`.

Now, we are going to create a vector to store these three values, and we do that by calling `VectorAssembler`. 

In [5]:
vecAssembler = VectorAssembler(inputCols=['cpu_utilization','free_memory','session_count'], outputCol='features')

Now, VectorAssembler returns a data structure, and then we will use this data structure to create a DataFrame by combining the mentioned columns into a single vector and put that vector in a new column called `features`.

In [6]:
vecUtil_df = vecAssembler.transform(df_util)

In [7]:
vecUtil_df.show(10)

+---------------+-------------------+-----------+---------+-------------+----------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|
+---------------+-------------------+-----------+---------+-------------+----------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.35,58.0]|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58| [0.41,0.4,58.0]|

Now, we want to use this DataFrame in our clustering algorithm, all combined into a single column called `features`. The reason we did this is because the Machine Learning algorithms in Spark MLLib expect the input data to be in a single vector. And now the ML algorithm, we are going to use is called **KMeans**.

In [9]:
# setK(3) number of clusters
# setSeed(1) it takes a seed for random value generation
kmeans = KMeans().setK(3).setSeed(1)

Now, `kmeans` is a data structure that is ready to run the KMeans algorithm. To do that, we will use `fit()`, and `fit()` is the command that is used to actually take input data and then apply the algorithm. 

In [10]:
kmModel=kmeans.fit(vecUtil_df)

The critical thing in a KMeans model is the cluster centers or centroids. So let's look up what the centroids are.

In [11]:
kmModel.clusterCenters()

[array([ 0.46901386,  0.52938041, 48.2003027 ]),
 array([ 0.75054326,  0.24974553, 90.25809685]),
 array([ 0.64528988,  0.35404374, 70.1378809 ])]

In [12]:
from pyspark.ml.evaluation import ClusteringEvaluator

In [13]:
pred_cluster_df = kmModel.transform(vecUtil_df)

In [14]:
pred_cluster_df.show()

+---------------+-------------------+-----------+---------+-------------+----------------+----------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|        features|prediction|
+---------------+-------------------+-----------+---------+-------------+----------------+----------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|[0.57,0.51,47.0]|         0|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|[0.47,0.62,43.0]|         0|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|[0.56,0.57,62.0]|         2|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|[0.57,0.56,50.0]|         0|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|[0.35,0.46,43.0]|         0|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|[0.41,0.58,48.0]|         0|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|[0.57,0.3

In [15]:
eval = ClusteringEvaluator()

In [16]:
SED_value = eval.evaluate(pred_cluster_df)

In [17]:
print("Squared Euclidean Distance Value = {:.2f}".format(SED_value))

Squared Euclidean Distance Value = 0.74


#### Well Done!